In [1]:
import pandas as pd
import fairness_metrics
import utils
import ast
import pickle
import operator
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def get_max_score(row):
    row = ast.literal_eval(row)
    # max_score = sum(list(row.values()))
    max_score = max(row.items(), key=operator.itemgetter(1))[1]
    return max_score

def get_max_class(row):
    row = regard_map[row]
    # row = ast.literal_eval(row)
    max_class = max(row.items(), key=operator.itemgetter(1))[0]
    return max_class

In [3]:
# gender based subset
gender_features = ['male', 'female', 'transgender', 'other_gender']
# race based subset
race_features = ['asian', 'black', 'latino', 'white', 'other_race_or_ethnicity']
# disability based subset
disability_features = ['intellectual_or_learning_disability', 'physical_disability', 'psychiatric_or_mental_illness', 'other_disability']
# sexual orientation subset
sexual_orientation_features = ['heterosexual', 'homosexual_gay_or_lesbian', 'bisexual', 'other_sexual_orientation']

labels = [gender_features, race_features, disability_features, sexual_orientation_features]
names = ['gender', 'race', 'disability', 'sexual_orientation']

In [4]:
# load the relevant datasets
ideology_df = pd.read_csv('./data/processed_annotated_comments.csv')
ideology_df['label'] = ideology_df['label'].apply(lambda x: None if x not in ['left', 'right'] else x)
ideology_df.dropna(inplace=True)

gender_df = pd.read_csv('./data/jigsaw/gender.csv')
race_df = pd.read_csv('./data/jigsaw/race.csv')
disability_df = pd.read_csv('./data/jigsaw/disability.csv')
sexual_orientation_df = pd.read_csv('./data/jigsaw/sexual_orientation.csv')

In [5]:
def make_moderated(df, moderation_map):
    moderations = []
    new_df = df.copy()
    for text in df['text']:
        if text in moderation_map.keys():
            moderations += [moderation_map[text]]
        else:
            new_df = new_df[new_df.text != text]

    df = new_df.copy()

    df['flagged'] = moderations

    return df

In [6]:
datasets = [ideology_df, gender_df, race_df, disability_df, sexual_orientation_df]
labels = [list(set(df['label'])) for df in datasets]
names = ['ideology', 'gender', 'race', 'disability', 'sexual orientation']
factors = ['positive', 'negative', 'neutral', 'other']

In [7]:
# cases to consider
label_map = {}
label_map['ideology'] = 'left' 
label_map['gender'] = 'male'
label_map['race'] = 'white'
label_map['disability'] = 'physical_disability'
label_map['sexual orientation'] = 'heterosexual'

In [8]:
def get_metrics(datasets, names, moderation_map):
    metrics = []
    for dataset, name in zip(datasets, names):
        dataset = make_moderated(dataset.copy(), moderation_map)
        subset = utils.process_dataset(dataset.copy(), label_map[name])
        # demographic parity
        dp = fairness_metrics.demographic_parity(df=subset.copy())
        
        # add regard
        regard = subset.copy()['text'].apply(get_max_class)

        csp = []
        for factor in factors:
            csp += [fairness_metrics.conditional_statistical_parity(df=subset.copy(), factors=regard, factor=factor)]
        csp += [dp, name, label_map[name]]
        metrics += [csp]
    columns = factors+['dp', 'bias', 'group']
    columns = [column.upper() for column in columns]
    df = pd.DataFrame(metrics, columns = columns)
    return df

In [9]:
# uncomment if you have saved the regard model results

# with open('./results/<regard_map_location>.pkl', 'rb') as handle: 3 comment_regard
    # regard_map = pickle.load(handle)

# uncomment if you have saved the moderation API results

# with open('./results/<openai_moderation_path>.pkl', 'rb') as handle: # comments_moderated_openai
    # moderation_map_openai = pickle.load(handle)

# with open('./results/<uniformly_random_baseline>.pkl', 'rb') as handle: # unirand_comments
    # moderation_map_unirand = pickle.load(handle)

# with open('./results/<clarifai_moderation_path>.pkl', 'rb') as handle: # comment_moderation_clarifai
    # moderation_map_clarifai = pickle.load(handle)

# with open('./results/<google_perspective_moderation_path>.pkl', 'rb') as handle: # comments_moderation_google_perspective
    # moderation_map_perspective = pickle.load(handle)

# with open('./results/<google_palm_moderation_path>.pkl', 'rb') as handle: # comments_moderation_google
    # moderation_map_google = pickle.load(handle)

In [10]:
# uncomment if processing raw moderation results
# moderation_map_openai = utils.process_openai(moderation_map_openai)
# moderation_map_clarifai = utils.process_clarifai(moderation_map_clarifai)
# moderation_map_perspective = utils.process_perspective(moderation_map_perspective)
# moderation_map_google = utils.process_google(moderation_map_google)

In [11]:
# load preprocessed results
with open('./results/moderation_results.pkl', 'rb') as file:
    fairness_results = pickle.load(file)

moderation_map_unirand = {k:v['unirand'] for k,v in fairness_results.items()}
moderation_map_openai = {k:v['openai'] for k,v in fairness_results.items()}
moderation_map_clarifai = {k:v['clarifai'] for k,v in fairness_results.items()}
moderation_map_perspective = {k:v['perspective'] for k,v in fairness_results.items()}
moderation_map_google = {k:v['google'] for k,v in fairness_results.items() if v['google'] in [True, False]}
regard_map = {k:v['regard'] for k,v in fairness_results.items()}

In [12]:
df_unirand = get_metrics(datasets=datasets.copy(), names=names, moderation_map=moderation_map_unirand)
df_openai = get_metrics(datasets=datasets.copy(), names=names, moderation_map=moderation_map_openai)
df_clarifai = get_metrics(datasets=datasets.copy(), names=names, moderation_map=moderation_map_clarifai)
df_google = get_metrics(datasets=datasets.copy(), names=names, moderation_map=moderation_map_google)
df_perspective = get_metrics(datasets=datasets.copy(), names=names, moderation_map=moderation_map_perspective)

In [ ]:
df_unirand

In [ ]:
df_openai

In [ ]:
df_clarifai

In [ ]:
df_google

In [ ]:
df_perspective

In [ ]:
# visualize the results
import fairness_visualization
import seaborn as sns
import matplotlib.patches as mpatches

sns.set_theme()
sns.set_style("ticks")
color_1 = (sns.color_palette("pastel")[5], sns.color_palette("pastel")[5], sns.color_palette("pastel")[5])
color_2 = (sns.color_palette("pastel")[0], sns.color_palette("pastel")[0], sns.color_palette("pastel")[0])
color_3 = (sns.color_palette("pastel")[-3], sns.color_palette("pastel")[-3], sns.color_palette("pastel")[-3])
color_4 = (sns.color_palette("pastel")[1], sns.color_palette("pastel")[1], sns.color_palette("pastel")[1])
color_5 = (sns.color_palette("pastel")[-1], sns.color_palette("pastel")[-1], sns.color_palette("pastel")[-1])
colors = [color_1, color_2, color_3, color_4, color_5]
dfs = [df_openai, df_perspective, df_google, df_clarifai]
names = ['A', 'B', 'C', 'D']
tuples_main_xy = [(a,b) for a,b in zip(df_unirand['NEGATIVE'], df_unirand['DP'])]

fig, ax = plt.subplots(2, 2, figsize=(4, 4))
ax = ax.flatten()
x_max = -1
y_max = -1
for i in range(0, len(ax)):
    tuples_xy = [(a,b) for a,b in zip(dfs[i]['NEGATIVE'], dfs[i]['DP'])]
    local_max_y = max(max(dfs[i]['DP'].tolist()), max(df_unirand['DP'].tolist()))
    local_max_x = max(max(dfs[i]['NEGATIVE'].tolist()), max(df_unirand['NEGATIVE'].tolist()))
    if  local_max_x > x_max:
        x_max = local_max_x
    if local_max_y > y_max:
        y_max = local_max_y


    fairness_visualization.plot_specific(ax[i], tuples_main_xy, tuples_xy, colors)
    ax[i].text(x=0.88, y=0.85, s=names[i], fontsize=13, weight="bold", transform=ax[i].transAxes)

for i in range(0, len(ax)):
    ax[i].set_xlim(0, min(x_max+0.05, 1))
    ax[i].set_ylim(0, min(y_max+0.05, 1))

markers = ['s', '^']
marker_texts = ['Baseline Classifier', 'ASM Classifier']
new_patches = [plt.plot([], [], marker=markers[i], ms=10, ls="", mec='k', mfc='w',
                label="{:s}".format(marker_texts[i]))[0] for i in range(2)]
legend1 = fig.legend(handles=new_patches, bbox_to_anchor=(0.54, 0.15),prop={'size': 11},
                        loc='center', ncol=5, handletextpad=1, labelspacing=1, columnspacing=0.5, framealpha=0)

ax[1].add_artist(legend1)

plt.tight_layout(pad=1)
fig.subplots_adjust(bottom=0.35)
colors = [color_1[0], color_2[0], color_3[0], color_4[0], color_5[0]]
texts = ["Ideology", "Gender", "Ethnicity", "Disability", "Sexual Orientation"]
patches = [mpatches.Patch(color=colors[i], label=texts[i]) for i in range(len(texts))]
fig.legend(handles=patches, bbox_to_anchor=(0.54, 0.025),prop={'size': 12},
loc='center', ncol=3, handletextpad=2, labelspacing=1, columnspacing=0.5, framealpha=0)


plt.show()